<a href="https://colab.research.google.com/github/AndrewRucker/Fantasy-Football-and-Betting/blob/main/BestBall/DK/DK-MillyMaker-Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Library imports and dataset creation

In [38]:
!pip install "ipython>=7"

In [39]:
import IPython
IPython.__version__

'7.33.0'

In [36]:
!pip install altair-viewer

     |████████████████████████████████| 844 kB 11.5 MB/s 


In [37]:
!pip install vega==3.6

In [121]:
import pandas as pd
import seaborn as sns
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.core.pylabtools import figsize
from IPython.display import display
sns.set_context(font_scale=2)
alt.renderers.enable('colab')


# to bypass warnings in various dataframe assignments
pd.options.mode.chained_assignment = None

In [122]:
#Read in CSV to dataframe
url = 'https://raw.githubusercontent.com/AndrewRucker/Fantasy-Football-and-Betting/main/BestBall/DK/DK5.csv'
df = pd.read_csv(url, index_col='Team')
df = df.dropna(how='all')

df

,POS,Name,TEAM,Round,Pick,ADP,Value,Draft Start,Cumulative Val,Strategy,Elite Onesie
Team,,,,,,,,,,,
1,RB,Christian Mccaffrey,CAR,1,2,2.6,-0.6,5/13/2022,94.3,Hyperfragile RB,Elite TE
1,TE,Mark Andrews,BAL,2,23,19.3,3.7,5/13/2022,94.3,Hyperfragile RB,Elite TE
1,RB,Saquon Barkley,NYG,3,26,25.7,0.3,5/13/2022,94.3,Hyperfragile RB,Elite TE
1,RB,James Conner,AZ,4,47,34.8,12.2,5/13/2022,94.3,Hyperfragile RB,Elite TE
1,RB,Cam Akers,LAR,5,50,43.5,6.5,5/13/2022,94.3,Hyperfragile RB,Elite TE
...,...,...,...,...,...,...,...,...,...,...,...
13,RB,Tyrion Davis-Price,SF,16,187,177.0,10.0,5/16/2022,60.6,Anchor RB,Elite QB/TE
13,RB,Brian Robinson,WAS,17,198,191.9,6.1,5/16/2022,60.6,Anchor RB,Elite QB/TE
13,RB,James White,NE,18,211,211.0,0.0,5/16/2022,60.6,Anchor RB,Elite QB/TE


#### Initial data manipulation

In [123]:
df1 = df.groupby('Team')['POS'].apply(lambda x: ', '.join(x.astype(str))).reset_index()

df1['RB'] = df1['POS'].str.count('RB')
df1['WR'] = df1['POS'].str.count('WR')
df1['QB'] = df1['POS'].str.count('QB')
df1['TE'] = df1['POS'].str.count('TE')

df1['build'] = df1['QB'].astype(str) + "/" + df1['RB'].astype(str) + "/" + df1['WR'].astype(str) + "/" + df1['TE'].astype(str)
df1 = df1.drop(columns=['POS', 'RB', 'WR', 'QB', 'TE'])
df1

,Team,build
0,1,3/5/10/2
1,2,2/5/10/3
2,3,2/5/10/3
3,4,2/7/8/3
4,5,2/6/9/3
5,6,2/7/9/2
6,7,2/7/8/3
7,8,3/4/9/4
8,9,3/5/10/2
9,10,3/6/8/3


In [124]:
df2 = pd.merge(df, df1, how='left', on = 'Team')
df3 = df2.groupby('Team')['Name'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
df2 = pd.merge(df2, df3, how='left', on='Team')

In [125]:
final_df = df2.drop(columns=['POS', 'Name_x', 'TEAM', 'Round', 'Pick', 'ADP', 'Value'])
final_df = final_df.drop_duplicates()
final_df ['Players'] = final_df['Name_y']
final_df = final_df.drop(columns=['Name_y'])
final_df

split_df = pd.concat([final_df['Team'], final_df['Players'].str.split(', ', expand=True)], axis=1)
split_df.rename({0 : 1, 1 : 2, 2 : 3, 3 : 4, 4 : 5, 5 : 6, 6 : 7, 7 : 8, 8 : 9, 9 : 10, 10 : 11, 11 : 12, 12 : 13, 13 : 14, 14 : 15, 15 : 16, 16 : 17, 17 : 18, 18 : 19, 19 : 20}, axis='columns', inplace=True)

In [126]:
df1 = pd.merge(df1, df2, how='left', on = 'Team')
df1 = df1.drop(columns=['POS', 'Name_x', 'TEAM', 'Round', 'Pick', 'ADP', 'Value', 'Name_y'])
df1

,Team,build_x,Draft Start,Cumulative Val,Strategy,Elite Onesie,build_y
0,1,3/5/10/2,5/13/2022,94.3,Hyperfragile RB,Elite TE,3/5/10/2
1,1,3/5/10/2,5/13/2022,94.3,Hyperfragile RB,Elite TE,3/5/10/2
2,1,3/5/10/2,5/13/2022,94.3,Hyperfragile RB,Elite TE,3/5/10/2
3,1,3/5/10/2,5/13/2022,94.3,Hyperfragile RB,Elite TE,3/5/10/2
4,1,3/5/10/2,5/13/2022,94.3,Hyperfragile RB,Elite TE,3/5/10/2
...,...,...,...,...,...,...,...
255,13,2/8/8/2,5/16/2022,60.6,Anchor RB,Elite QB/TE,2/8/8/2
256,13,2/8/8/2,5/16/2022,60.6,Anchor RB,Elite QB/TE,2/8/8/2
257,13,2/8/8/2,5/16/2022,60.6,Anchor RB,Elite QB/TE,2/8/8/2
258,13,2/8/8/2,5/16/2022,60.6,Anchor RB,Elite QB/TE,2/8/8/2


#### Player/Build viz and stats

In [127]:
df2

,Team,POS,Name_x,TEAM,Round,Pick,ADP,Value,Draft Start,Cumulative Val,Strategy,Elite Onesie,build,Name_y
0,1,RB,Christian Mccaffrey,CAR,1,2,2.6,-0.6,5/13/2022,94.3,Hyperfragile RB,Elite TE,3/5/10/2,"Christian Mccaffrey, Mark Andrews, Saquon Bark..."
1,1,TE,Mark Andrews,BAL,2,23,19.3,3.7,5/13/2022,94.3,Hyperfragile RB,Elite TE,3/5/10/2,"Christian Mccaffrey, Mark Andrews, Saquon Bark..."
2,1,RB,Saquon Barkley,NYG,3,26,25.7,0.3,5/13/2022,94.3,Hyperfragile RB,Elite TE,3/5/10/2,"Christian Mccaffrey, Mark Andrews, Saquon Bark..."
3,1,RB,James Conner,AZ,4,47,34.8,12.2,5/13/2022,94.3,Hyperfragile RB,Elite TE,3/5/10/2,"Christian Mccaffrey, Mark Andrews, Saquon Bark..."
4,1,RB,Cam Akers,LAR,5,50,43.5,6.5,5/13/2022,94.3,Hyperfragile RB,Elite TE,3/5/10/2,"Christian Mccaffrey, Mark Andrews, Saquon Bark..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,13,RB,Tyrion Davis-Price,SF,16,187,177.0,10.0,5/16/2022,60.6,Anchor RB,Elite QB/TE,2/8/8/2,"Jamar Chase, Mark Andrews, Javonte Williams, L..."
256,13,RB,Brian Robinson,WAS,17,198,191.9,6.1,5/16/2022,60.6,Anchor RB,Elite QB/TE,2/8/8/2,"Jamar Chase, Mark Andrews, Javonte Williams, L..."
257,13,RB,James White,NE,18,211,211.0,0.0,5/16/2022,60.6,Anchor RB,Elite QB/TE,2/8/8/2,"Jamar Chase, Mark Andrews, Javonte Williams, L..."
258,13,WR,Danny Gray,SF,19,222,222.0,0.0,5/16/2022,60.6,Anchor RB,Elite QB/TE,2/8/8/2,"Jamar Chase, Mark Andrews, Javonte Williams, L..."


In [128]:
#Exposure calc
exp_df = df2['Name_x'].value_counts()
exp_df = exp_df.reset_index()
exp_df['Count'] = exp_df['Name_x']
exp_df['%'] = round((exp_df['Count'] / df2['Team'].max()) * 100, 1)
exp_df = exp_df.drop(columns=['Name_x'])
exp_df.rename({'index' : 'Player'}, axis='columns', inplace=True)
pos_df = df2.drop_duplicates(subset='Name_x')
pos_df = pos_df.drop(columns=['Team', 'TEAM', 'Round', 'Pick', 'ADP', 'Value', 'Draft Start', 'Cumulative Val', 'Strategy', 'Elite Onesie', 'build', 'Name_y'])
pos_df.rename({'Name_x' : 'Player'}, axis='columns', inplace=True)
exp_df = exp_df.merge(pos_df, on='Player', how='left')
exp_df

,Player,Count,%,POS
0,Rachaad White,7,53.8,RB
1,Trey Lance,6,46.2,QB
2,Chris Evans,5,38.5,RB
3,Hunter Renfrow,5,38.5,WR
4,JuJu Smith-Schuster,5,38.5,WR
...,...,...,...,...
139,James Robinson,1,7.7,RB
140,Robert Woods,1,7.7,WR
141,Devonta Smith,1,7.7,WR
142,Mike Evans,1,7.7,WR


In [129]:
pos_count = exp_df['POS'].value_counts()
pos_count = pos_count.reset_index()
pos_count = pos_count.rename({'index' : 'Position'}, axis='columns', inplace=True)
pos_count

In [130]:

#Player Exposure
alt.Chart(exp_df).mark_circle(size=60).encode(
    x='Count',
     y=alt.Y("Player", sort='-x'),
    color='Count',
    tooltip=['Player', 'Count', '%']
).interactive()

alt.Chart(...)

In [131]:
#Better exp viz
#my_range=range(1,len(exp_df.index)+1)

#plt.stem(exp_df['Count'])
#plt.xticks(my_range, exp_df['Player'])

In [132]:
#Team Build by draft date, with cumulative ADP value

alt.Chart(df1).mark_circle(size=40).encode(
    x='Draft Start',
    y='Cumulative Val',
    color='build_x',
    tooltip=['Draft Start','Team', 'Strategy', 'Elite Onesie', 'build_x', 'Cumulative Val']
).interactive()

alt.Chart(...)

In [133]:
#Team Build by draft date, with cumulative ADP value

alt.Chart(df1).mark_circle(size=40).encode(
    x='Draft Start',
    y='Cumulative Val',
    color='Strategy',
    tooltip=['Draft Start','Team', 'Strategy', 'Elite Onesie', 'build_x', 'Cumulative Val']
).interactive()

alt.Chart(...)

In [134]:
#Elite QB/TE with CLV/Date
alt.Chart(df1).mark_circle(size=40).encode(
    x='Draft Start',
    y='Cumulative Val',
    color='Elite Onesie',
    tooltip=['Draft Start','Team', 'Strategy', 'Elite Onesie', 'build_x', 'Cumulative Val']
).interactive

<bound method Chart.interactive of alt.Chart(...)>

In [135]:
#Breakdown of all builds
build_count = final_df['build'].value_counts()
build_count = build_count.reset_index()
build_count['Count'] = build_count['build']
build_count['%'] = round((build_count['Count'] / df2['Team'].max()) * 100, 1)
build_count = build_count.drop(columns=['build'])
build_count.rename({'index' : 'Build'}, axis='columns', inplace=True)
build_count

,Build,Count,%
0,2/7/8/3,3,23.1
1,3/5/10/2,2,15.4
2,2/5/10/3,2,15.4
3,2/7/9/2,2,15.4
4,2/6/9/3,1,7.7
5,3/4/9/4,1,7.7
6,3/6/8/3,1,7.7
7,2/8/8/2,1,7.7


In [136]:
#Breakdown of structural strategies
strat_count = final_df['Strategy'].value_counts()
strat_count = strat_count.reset_index()
strat_count['Count'] = strat_count['Strategy']
strat_count['%'] = round((strat_count['Count'] / df2['Team'].max()) * 100, 1)
strat_count = strat_count.drop(columns=['Strategy'])
strat_count.rename({'index' : 'Strategy'}, axis='columns', inplace=True)
strat_count

,Strategy,Count,%
0,Anchor RB,7,53.8
1,Hyperfragile RB,2,15.4
2,Zero RB,2,15.4
3,Superhero RB,2,15.4


In [137]:
#Breakdown of Elite QB/TE teams
onesie_count = final_df['Elite Onesie'].value_counts()
onesie_count = onesie_count.reset_index()
onesie_count['Count'] = onesie_count['Elite Onesie']
onesie_count['%'] = round((onesie_count['Count'] / df2['Team'].max()) * 100, 1)
onesie_count = onesie_count.drop(columns=['Elite Onesie'])
onesie_count.rename({'index' : 'Elite Onesie'}, axis='columns', inplace=True)
onesie_count

,Elite Onesie,Count,%
0,Neither,5,38.5
1,Elite TE,3,23.1
2,Elite QB,3,23.1
3,Elite QB/TE,2,15.4


#### Player team by round search

In [163]:
#Function that returns all teams with player passed, along with the builds used with the player
def player_team_search(players):
      base = r'^{}'
      expr = '(?=.*{})'
      base = base.format(''.join(expr.format(p) for p in players))
      #Change for number of rounds
      rounds = 20
      i = 1

      with_player = final_df.loc[final_df['Players'].str.contains(base)]
      with_player = with_player.drop_duplicates(subset=['Team'])

      #Building out round by round breakdown for teams with given player
      split_df = pd.concat([final_df['Team'], final_df['Players'].str.split(', ', expand=True)], axis=1)
      split_df.rename({0 : 1, 1 : 2, 2 : 3, 3 : 4, 4 : 5, 5 : 6, 6 : 7, 7 : 8, 8 : 9, 9 : 10, 10 : 11, 11 : 12, 12 : 13, 13 : 14, 14 : 15, 15 : 16, 16 : 17, 17 : 18, 18 : 19, 19: 20}, axis='columns', inplace=True)
      split_df = split_df.merge(with_player, on=['Team', 'Team'], how='right')
      split_df = split_df.drop_duplicates(subset=['Team'])
      print("Round by Round selections with " + ', '.join(players) + " on the team \n")
      while i <= rounds:
        round_count = split_df[i].value_counts().rename_axis('Player').reset_index(name='Times Selected')
        round_count = round_count.sort_index()
        print("===============================================================")
        print("                                    Round " + str(i) + " \n")
        chart = alt.Chart(round_count).mark_bar().encode(
            x=alt.X('Times Selected', axis=alt.Axis(labels=True), title=""),
            y=alt.Y('Player', axis=alt.Axis(labels=True), sort='-x', title=""),
            tooltip=['Player', 'Times Selected'],
            color=alt.Color('Player', legend=None, scale=alt.Scale(scheme=alt.SchemeParams(name='darkblue')))
        ).interactive()
        chart.display(renderer='png')
        i=i+1

      #List of teams with given player on roster
      #print("===========================================================")
      #print("Full Teams with " + ', '.join(players) + " on the roster \n")
      #print(with_player['Players'].value_counts())

#### Build Count with Given Players

In [162]:
#Build Count with given players function
def builds_with_players(players):
      base = r'^{}'
      expr = '(?=.*{})'
      base = base.format(''.join(expr.format(p) for p in players))
      #Change for number of rounds
      rounds = 20
      i = 1

      with_player = final_df.loc[final_df['Players'].str.contains(base)]
      with_player = with_player.drop_duplicates(subset=['Team'])    
      print("Structural builds with " + ', '.join(players) + " on the roster")
      print(with_player['build'].value_counts())
      chart=alt.Chart(with_player).mark_circle(size=30).encode(
      x='Draft Start',
      y='Cumulative Val',
      color='build',
      tooltip=['Draft Start','Team', 'build', 'Cumulative Val']
      ).interactive()
      chart.display(renderer='png')
      print("\n")

      

#### Strategy and Elite Onesie count functions

In [160]:
#Strat Count with given players function
def strats_with_players(players):
      base = r'^{}'
      expr = '(?=.*{})'
      base = base.format(''.join(expr.format(p) for p in players))
      #Change for number of rounds
      rounds = 20
      i = 1

      with_player = final_df.loc[final_df['Players'].str.contains(base)]
      with_player = with_player.drop_duplicates(subset=['Team'])    
      print("Strategies with " + ', '.join(players) + " on the roster")
      print(with_player['Strategy'].value_counts())
      chart=alt.Chart(with_player).mark_circle(size=30).encode(
      x='Draft Start',
      y='Cumulative Val',
      color='Strategy',
      tooltip=['Draft Start','Team', 'Strategy', 'build', 'Cumulative Val']
      ).interactive()
      chart.display(renderer='png')
      print("\n")

In [186]:
#Elite Onesie Count with given players function
def onesie_with_players(players):
      base = r'^{}'
      expr = '(?=.*{})'
      base = base.format(''.join(expr.format(p) for p in players))
      #Change for number of rounds
      rounds = 20
      i = 1

      with_player = final_df.loc[final_df['Players'].str.contains(base)]
      with_player = with_player.drop_duplicates(subset=['Team'])    
      print("Elite Onesie teams with " + ', '.join(players) + " on the roster")
      print(with_player['Elite Onesie'].value_counts())
      chart=alt.Chart(with_player).mark_circle(size=30).encode(
      x='Draft Start',
      y='Cumulative Val',
      color='Elite Onesie',
      tooltip=['Draft Start','Team', 'Strategy', 'Elite Onesie', 'build', 'Cumulative Val']
      ).interactive()
      chart.display(renderer='png')
      print("\n")

#### To use while drafting

In [187]:
#Use of build and player search functions
players_to_search = ['Jonathan Taylor']
#builds_with_players(players_to_search)
#strats_with_players(players_to_search)
onesie_with_players(players_to_search)


Elite Onesie teams with Jonathan Taylor on the roster
Neither    2
Name: Elite Onesie, dtype: int64


alt.Chart(...)

In [165]:
#Round by Round searching
player_team_search(players_to_search)

Round by Round selections with Jonathan Taylor on the team 

                                    Round 1 



alt.Chart(...)

                                    Round 2 



alt.Chart(...)

                                    Round 3 



...

alt.Chart(...)

                                    Round 19 



alt.Chart(...)

                                    Round 20 



alt.Chart(...)

#### Round by round ADP reaches (work in progress)

In [144]:
#General ADP Guidelines. This function when given a pick will give the user the likely players availible to them in each round, given current ADP.

#WORK IN PROGRESS



def adp_dist(pick):
  adp_url = 'https://raw.githubusercontent.com/AndrewRucker/Fantasy-Football-and-Betting/main/BestBall/Underdog/adps.csv'
  adp = pd.read_csv(adp_url, index_col='Player ID')
  adp = adp.dropna(how='all')
  adp = adp.drop_duplicates(subset=['Player Name'])
  picks_url = 'https://raw.githubusercontent.com/AndrewRucker/Fantasy-Football-and-Betting/main/BestBall/PicksByRound.csv'
  possible_picks = ['Round','1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th', '11th', '12th' ]
  picks = pd.read_csv(picks_url, index_col='Round', names=possible_picks, encoding="utf-8-sig")
  picks = picks.dropna(how='all')
  picks = picks.iloc[1: , :]
  picks   
  round = 0
  rounds = 19
  team_temp = pd.DataFrame(columns=['Team', 'Diff'])
  while round <= rounds:
    pos = picks.iloc[round].loc[pick]
    plus = int(pos) + 5
    minus = int(pos) - 5
    temp = adp[adp['ADP'].between(minus, plus)]
    temp['diff'] = int(pos) - temp['ADP']
    error_bars = alt.Chart(temp).mark_errorbar(extent='stdev').encode(
    x=alt.X('diff', scale=alt.Scale(zero=False)),
    y=alt.Y('Player Name')
    )

    points = alt.Chart(temp).mark_point(filled=True, color='black').encode(
    x=alt.X('diff', aggregate='mean'),
    y=alt.Y('Player Name'),
    tooltip=['Player Name','diff', 'ADP', 'Team']
    ).interactive()

    ye = error_bars + points
    ye.display() 
    round = round + 1



  


#### Most likely Team Stacks by pick position accoridng to ADP. (work in progress)

In [145]:
#This function will tell you the most likely team stacks given a draft position for players with an ADP lower than 180.
#The closer to zero a team's value is, the more likely that team stack from that draft position has happened.
#General ADP Guidelines. This function when given a pick will give the user the likely players availible to them in each round, given current ADP.
def Likely_team_stacks(pick):
  adp_url = 'https://raw.githubusercontent.com/AndrewRucker/Fantasy-Football-and-Betting/main/BestBall/Underdog/adps.csv'
  adp = pd.read_csv(adp_url, index_col='Player ID')
  adp = adp.dropna(how='all')
  adp = adp.drop_duplicates(subset=['Player Name'])
  picks_url = 'https://raw.githubusercontent.com/AndrewRucker/Fantasy-Football-and-Betting/main/BestBall/PicksByRound.csv'
  possible_picks = ['Round','1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th', '11th', '12th' ]
  picks = pd.read_csv(picks_url, index_col='Round', names=possible_picks, encoding="utf-8-sig")
  picks = picks.dropna(how='all')
  picks = picks.iloc[1: , :]
  picks   
  round = 0
  rounds = 19
  team_temp = pd.DataFrame(columns=['Team', 'Diff'])
  temp1 = adp[adp['ADP'] <= 190]
  while round <= rounds:
    pos = picks.iloc[round].loc[pick]
    plus = int(pos) + 5
    minus = int(pos) - 5
    temp = temp1[temp1['ADP'].between(minus, plus)]
    temp['diff'] = int(pos) - temp['ADP']
    for index, row in temp.iterrows():
      team_temp = team_temp.append({'Team': row['Team'], 'Diff' : row['diff']}, ignore_index=True)
    
    round = round + 1
  Avg_team_diff = team_temp.groupby(['Team'])
  Avg_team_diff = team_temp.groupby(['Team'])['Diff'].agg(lambda x: x.unique().mean())
  Avg_team_diff = Avg_team_diff.sort_values(ascending=False)
  print(Avg_team_diff)
  #print(temp1.shape)
  



#### Correlation between pick position and team stack by ADP (work in progress)

In [146]:
#This function will tell you the most likely team stacks given a draft position for players with an ADP lower than 180.
#The closer to zero a team's value is, the more likely that team stack from that draft position has happened.
#General ADP Guidelines. This function when given a pick will give the user the likely players availible to them in each round, given current UD ADP.
def pick_to_stack_corr():
  adp_url = 'https://raw.githubusercontent.com/AndrewRucker/Fantasy-Football-and-Betting/main/BestBall/adps.csv'
  adp = pd.read_csv(adp_url, index_col='Player ID')
  adp = adp.dropna(how='all')
  adp = adp.drop_duplicates(subset=['Player Name'])
  picks_url = 'https://raw.githubusercontent.com/AndrewRucker/Fantasy-Football-and-Betting/main/BestBall/PicksByRound.csv'
  possible_picks = ['Round','1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th', '11th', '12th']
  picks = pd.read_csv(picks_url, index_col='Round', names=possible_picks, encoding="utf-8-sig")
  picks = picks.dropna(how='all')
  picks = picks.iloc[1: , :]
  picks   
  round = 0
  rounds = 19
  avg_team_df = pd.DataFrame(columns=['Team', 'Avg'])
  for i in possible_picks:
    team_temp = pd.DataFrame(columns=['Team', 'Diff'])
    temp1 = adp[adp['ADP'] <= 190]
    while round <= rounds:
      pos = picks.iloc[round].loc[i]
      plus = int(pos) + 5
      minus = int(pos) - 5
      temp = temp1[temp1['ADP'].between(minus, plus)]
      temp['diff'] = int(pos) - temp['ADP']
      for index, row in temp.iterrows():
        team_temp = team_temp.append({'Team': row['Team'], 'Diff' : row['diff']}, ignore_index=True)
      round = round + 1
    Avg_team_diff = team_temp.groupby(['Team'])
    Avg_team_diff = team_temp.groupby(['Team'])['Diff'].agg(lambda x: x.unique().mean())
    Avg_team_diff = Avg_team_diff.sort_values(ascending=False)
    for index, row in Avg_team_diff.iterrows():
      avg_team_df = avg_team_df.append({'Team' : row['Team'], 'Avg' : row['Diff']}. ignore_index=True)

    print(Avg_team_diff)
    #print(temp1.shape)
  



SyntaxError: ignored

#### Functions to call ADP insights (work in progress)

In [ ]:
#Insert pick you have to see round by round breakdown of the players availible to you.
#pick = '6th'
#adp_dist(pick)
#Likely_team_stacks(pick)

In [ ]:
  picks_url = 'https://raw.githubusercontent.com/AndrewRucker/Fantasy-Football-and-Betting/main/BestBall/PicksByRound.csv'
  possible_picks = ['Round','1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th', '11th', '12th' ]
  picks = pd.read_csv(picks_url, index_col='Round', names=possible_picks, encoding="utf-8-sig")
  picks = picks.dropna(how='all')
  picks = picks.iloc[1: , :]
  picks   